## Creating the logistic regression to predict absenteenism

### Import the relevant libraries

In [3]:
import pandas as pd
import numpy as np

### Load the data

In [5]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [6]:
data_preprocessed.head(5)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,1,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


### Create the target

In [8]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [9]:
# sử dụng np.where(điều kiện, giá trị trả về 1, giá trị trả về 2) giống như if else
target = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0) 

In [10]:
data_preprocessed['Excessive Absenteeism'] = target

In [11]:
data_preprocessed.head(5)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,1,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


### A comment on target

In [13]:
# 45% mang giá trị 1, 55% mang giá trị 0
target.sum() / target.shape[0]

0.45571428571428574

In [14]:
data_with_target = data_preprocessed.copy()

In [15]:
data_with_target = data_with_target.drop(['Absenteeism Time in Hours', 'Day of the week',
                                            'Daily Work Load Average','Distance to Work'], axis = 1)

In [16]:
data_with_target is data_preprocessed

False

In [17]:
data_with_target.head(5)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,1,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


### Select the inputs for the regression

In [19]:
data_with_target.shape

(700, 12)

In [20]:
data_with_target.iloc[:, : -1]

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,289,33,30,0,2,1
1,0,0,0,0,7,118,50,31,0,1,0
2,0,0,0,1,7,179,38,31,0,0,0
3,1,0,0,1,7,279,39,24,0,2,0
4,0,0,0,1,7,289,33,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,1,5,179,40,22,1,2,0
696,1,0,0,1,5,225,28,24,0,1,2
697,1,0,0,1,5,330,28,25,1,0,0
698,0,0,0,1,5,235,32,25,1,0,0


In [21]:
unscale_data = data_with_target.iloc[:, : -1]

### Standardize the data

In [23]:
from sklearn.preprocessing import StandardScaler

absenteeism_scale = StandardScaler()

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, columns, copy = True, with_mean = True, with_std = True):
        self.scaler = StandardScaler(copy = copy, with_mean = with_mean, with_std = with_std)
        self.columns = columns
        self.copy = copy
        self.with_mean = with_mean
        self.with_std = with_std
        self.mean_ = None
        self.var_ = None 

    def fit(self, X, y = None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns], axis = 0)
        self.var_ = np.var(X[self.columns], axis = 0)
        return self

    def transform(self, X, y = None, copy = None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns = self.columns)
        X_not_scaled = X.loc[:, ~X.columns.isin(self.columns)]
        return pd.concat([X_scaled, X_not_scaled], axis = 1)[init_col_order]

In [25]:
unscale_data.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [26]:
# columns_to_scaled = ['Month Value', 'Day of the week', 'Transportation Expense', 'Distance to Work',
#        'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
#        'Children', 'Pets']

columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [27]:
columns_to_scale = [x for x in unscale_data.columns.values if x not in columns_to_omit]

In [28]:
absenteeism_scale = CustomScaler(columns_to_scale)

In [29]:
absenteeism_scale.fit(unscale_data)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [30]:
scale_inputs = absenteeism_scale.transform(unscale_data)

In [31]:
scale_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,1,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,1,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,1,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,1,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


### Split the data into train & test and shuffle

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_test, y_train, y_test = train_test_split(scale_inputs, target, train_size = 0.8, random_state = 20)

In [35]:
print(x_train.shape, y_train.shape)

(560, 11) (560,)


In [36]:
print(x_test.shape, y_test.shape)

(140, 11) (140,)


### Logistic Regresstion with Sklearn

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#### Training model

In [40]:
reg = LogisticRegression()

In [41]:
reg.fit(x_train, y_train)

LogisticRegression()

In [42]:
reg.score(x_train, y_train)

0.7875

### Manually check the accuracy

In [44]:
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [45]:
np.sum(model_outputs == y_train)

441

In [46]:
model_outputs.shape[0]

560

In [47]:
np.sum(model_outputs == y_train) / model_outputs.shape[0]

0.7875

### Finding the intercept and coefficients

In [49]:
#coef_ → chứa các hệ số (weights): [w₁, w₂, ..., wₙ]

#intercept_ → là hệ số chặn (bias): b

reg.intercept_

array([-3.21683108])

In [50]:
reg.coef_

array([[ 1.98238129,  0.49528992,  2.37877814,  2.45317221,  0.17998465,
         0.66250801, -0.17799445,  0.30628123, -0.24506209,  0.37075429,
        -0.30195521]])

In [51]:
unscale_data.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [52]:
feature_name = unscale_data.columns.values

In [53]:
summary_table = pd.DataFrame(columns = ['Feature_Name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table

,Feature_Name,Coefficients
0,Reason_1,1.982381
1,Reason_2,0.495290
2,Reason_3,2.378778
3,Reason_4,2.453172
4,Month Value,0.179985
5,Transportation Expense,0.662508
6,Age,-0.177994
7,Body Mass Index,0.306281
8,Education,-0.245062
9,Children,0.370754


In [54]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature_Name,Coefficients
0,Intercept,-3.216831
1,Reason_1,1.982381
2,Reason_2,0.495290
3,Reason_3,2.378778
4,Reason_4,2.453172
5,Month Value,0.179985
6,Transportation Expense,0.662508
7,Age,-0.177994
8,Body Mass Index,0.306281
9,Education,-0.245062


### Interpreting the coefficients

In [56]:
summary_table['Odds_radio'] = np.exp(summary_table.Coefficients)

In [57]:
summary_table

,Feature_Name,Coefficients,Odds_radio
0,Intercept,-3.216831,0.040082
1,Reason_1,1.982381,7.260011
2,Reason_2,0.495290,1.640974
3,Reason_3,2.378778,10.791709
4,Reason_4,2.453172,11.625166
5,Month Value,0.179985,1.197199
6,Transportation Expense,0.662508,1.939651
7,Age,-0.177994,0.836947
8,Body Mass Index,0.306281,1.358364
9,Education,-0.245062,0.782656


In [58]:
# sort_values: sắp xếp từ nhỏ đến lớn
# ascending = False: sắp xếp trừ lớn -> nhỏ
summary_table.sort_values('Odds_radio', ascending = False)

,Feature_Name,Coefficients,Odds_radio
4,Reason_4,2.453172,11.625166
3,Reason_3,2.378778,10.791709
1,Reason_1,1.982381,7.260011
6,Transportation Expense,0.662508,1.939651
2,Reason_2,0.495290,1.640974
10,Children,0.370754,1.448827
8,Body Mass Index,0.306281,1.358364
5,Month Value,0.179985,1.197199
7,Age,-0.177994,0.836947
9,Education,-0.245062,0.782656


### Testing Model

In [60]:
reg.score(x_test, y_test)

0.7357142857142858

In [61]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.70232218, 0.29767782],
       [0.56341257, 0.43658743],
       [0.40137651, 0.59862349],
       [0.78065367, 0.21934633],
       [0.07295522, 0.92704478],
       [0.32895802, 0.67104198],
       [0.29427206, 0.70572794],
       [0.11112448, 0.88887552],
       [0.78952607, 0.21047393],
       [0.74344088, 0.25655912],
       [0.46342113, 0.53657887],
       [0.19069918, 0.80930082],
       [0.05817352, 0.94182648],
       [0.73034324, 0.26965676],
       [0.28839181, 0.71160819],
       [0.52659772, 0.47340228],
       [0.52756195, 0.47243805],
       [0.5147388 , 0.4852612 ],
       [0.40034309, 0.59965691],
       [0.04120345, 0.95879655],
       [0.69172517, 0.30827483],
       [0.78065367, 0.21934633],
       [0.41922746, 0.58077254],
       [0.41922746, 0.58077254],
       [0.22954727, 0.77045273],
       [0.74367044, 0.25632956],
       [0.48229988, 0.51770012],
       [0.86380877, 0.13619123],
       [0.17940297, 0.82059703],
       [0.78065367, 0.21934633],
       [0.

In [62]:
predicted_proba.shape

(140, 2)

In [63]:
predicted_proba[:, 1]

array([0.29767782, 0.43658743, 0.59862349, 0.21934633, 0.92704478,
       0.67104198, 0.70572794, 0.88887552, 0.21047393, 0.25655912,
       0.53657887, 0.80930082, 0.94182648, 0.26965676, 0.71160819,
       0.47340228, 0.47243805, 0.4852612 , 0.59965691, 0.95879655,
       0.30827483, 0.21934633, 0.58077254, 0.58077254, 0.77045273,
       0.25632956, 0.51770012, 0.13619123, 0.82059703, 0.21934633,
       0.38684988, 0.68202316, 0.69331779, 0.56201703, 0.21934633,
       0.56328496, 0.21914025, 0.75272748, 0.42568658, 0.63640044,
       0.21067406, 0.43143242, 0.23723318, 0.13085759, 0.84646574,
       0.57718867, 0.69414934, 0.29767782, 0.22102526, 0.20225584,
       0.57181303, 0.09975482, 0.67104198, 0.27860794, 0.84907414,
       0.46201118, 0.90176851, 0.22896254, 0.10375196, 0.10862858,
       0.70387059, 0.65960044, 0.2984967 , 0.81272373, 0.2034948 ,
       0.27664694, 0.01834418, 0.21914025, 0.75381738, 0.31199367,
       0.21914025, 0.08551395, 0.91934655, 0.48936788, 0.63710

### Save the model

In [65]:
import pickle

In [66]:
# with open(file name, kiểu ghi), dump: lưu model, reg: đối tượng được kết suất
# pickle the model file
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [67]:
# pickle the scaler file
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scale, file)

In [68]:
with open('scaler', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)
type(scaler)

__main__.CustomScaler